## Functionality that needs to be added

- ~~Trim Raw Input~~
- ~~Night vs Day bouts (critera - choose where over 50% of bout is spent, in case of tie choose night)~~
- Create Sleep Profile and summary stats
- Write to seperate sheets in Excel

### Import Libraries that will be needed

In [1]:
import pandas as pd
import datetime

### Import data from .csv

In [2]:
df_raw = pd.read_csv("Bethany_Raw.csv")

### Trim to 24 hour range

In [3]:
lower_limit = datetime.time(10,0,0)
upper_limit = datetime.time(10,5,0)
def get_data_range(L, lower_limit, upper_limit):
    """
    Find the start and end indices for a 24 hour range starting between a given time range
    Takes arguments L - a list of datetime.time objects, lower_limit and upper_limit - datetime.time objects 
    to specificy the range in which to choose the start time
    Returns a tuple: (start_index, end_index)
    """
    start_index = 0
    for i in range(len(L)):
        if L[i] >= lower_limit and L[i] < upper_limit:
            start_index = i
            break
    return (start_index, start_index + 288)


In [4]:
# Test trim on raw data
df_raw.Time = pd.to_datetime(df_raw.Time, format='%H:%M:%S').dt.time
trim_index = get_data_range(list(df_raw.Time), lower_limit, upper_limit)
df_trim = df_raw[trim_index[0]:trim_index[1]]
df_trim = df_trim.reset_index(drop=True)

### Make a column of bools to indicate day vs night

In [5]:
def isNight(x):
    night_start = datetime.time(22,0,0)
    night_end = datetime.time(10,0,0)
    if (x >= night_start and x < datetime.time(23, 59, 59)) or x < night_end:
        return True
    else:
        return False

In [6]:
df_trim.loc[:,'isNight'] = df_trim.Time.apply(isNight)

### Generate bout indices

In [7]:
# Script for collecting bouts
# Will need to create a list of bout indices to access resp data during bouts
def get_bout_indices(L):
    """
    Takes a list L and returns a list of tuples of the start/end indices in which sleeping bouts occurred.
    I.e. if two sleeping bouts occured, the first from index 5 to 20, and the second from index 30 to 40,
    this function will return [(5,20), (30,40)]
    """
    indices = []
    start_index = 1
    end_index = 1
    in_bout = False
    
    for i in range(len(L)):
        if L[i] == 0 and in_bout == False: 
            start_index = i
            in_bout = True
        if (L[i] != 0 or i == len(L)-1) and in_bout == True:
            end_index = i
            in_bout = False
            if i == len(L)-1:
                indices.append((start_index, end_index+1))
            else:
                indices.append((start_index, end_index))
    return indices 
        

In [8]:
get_bout_indices(list(df_trim.iloc[:,11]))

[(61, 63),
 (65, 92),
 (96, 98),
 (99, 102),
 (109, 112),
 (113, 121),
 (123, 136),
 (151, 153),
 (154, 174),
 (177, 180),
 (183, 185),
 (188, 189),
 (190, 191),
 (192, 197),
 (198, 200),
 (201, 203),
 (204, 205),
 (206, 209),
 (210, 214),
 (215, 216),
 (217, 223),
 (225, 227),
 (228, 236),
 (239, 250),
 (251, 258),
 (259, 267),
 (268, 275),
 (277, 281),
 (282, 288)]

In [9]:
list(df_trim.iloc[:,4])[282:288]

[1.2434120000000002,
 1.2348139999999999,
 1.1852959999999999,
 1.140252,
 1.1358409999999999,
 1.144841]

### Get day bouts and night bouts

In [17]:
# Need to put this into a function, and make a loop to generate day/night bouts for each fly of 5 in the df
def get_day_night_bouts(df, resp_colnum, sleep_colnum):
    bout_indicies = get_bout_indices(list(df.iloc[:,sleep_colnum]))
    resp_bouts_day = []
    resp_bouts_night = []
    for i in bout_indicies:
        if df.isNight[i[0]:i[1]].mean() >= 0.5:
            resp_bouts_night.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
        else:
            resp_bouts_day.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
    return resp_bouts_day, resp_bouts_night

In [18]:
fly1_bouts = get_day_night_bouts(df_trim, 4,11)

In [19]:
fly1_bouts[1]

[[0.85789090000000001, 0.90719620000000001],
 [0.86505089999999996,
  0.89445400000000008,
  0.9370075000000001,
  0.91154709999999994,
  0.90682649999999998,
  0.9454458,
  0.91049430000000009,
  0.86561049999999995,
  0.87170170000000002,
  0.8504896999999999,
  0.86692659999999999,
  0.85906460000000007,
  0.90022640000000009,
  0.86232260000000005,
  0.87401369999999989,
  0.84854390000000002,
  0.85547329999999999,
  0.84415949999999995,
  0.85118440000000006,
  0.90393050000000008],
 [0.81070410000000004, 0.84026119999999993, 0.84216540000000006],
 [0.88346559999999985, 0.84457309999999997],
 [0.91185919999999987],
 [0.86056630000000001],
 [1.2458659999999999, 1.3135059999999998, 1.260168, 1.257457, 1.25207],
 [1.348179, 1.3587149999999999],
 [1.2810170000000001, 1.289906],
 [1.3265719999999999],
 [1.2916370000000001, 1.314786, 1.3047],
 [1.332193, 1.293156, 1.315069, 1.2358120000000001],
 [1.2810159999999999],
 [1.2455889999999998,
  1.293876,
  1.228817,
  1.196723,
  1.2558940

## Export to .csv

In [51]:
# Now we save this list of lists as a csv
resp_bouts_df = pd.DataFrame(resp_bouts).transpose()
resp_bouts_df.to_csv('test.csv', index=False, header=False)